## TF Framework
#### Manual implementation of TF in details - Target Code in the bottom

In [1]:
import numpy as np

We'll create a set of classes to implement the TF framework manually

Our goal is to use this framework to perform a basic matrix multiplication using the basic neural network structure of inputs, weights and nodes.
And make this code work:
    - 
      sess = Session()
      
      g = Graph()
      g.set_as_default()
      
      A = Variable([[10,20],[30,40]])
      b = Variable([1,1])
      
      x = Placeholder()
      
      y = matmul(A,x)
      
      sess.run(operation = z, feed_dict={x:10})


#### Building Operations

In [12]:
#Operation Class will not be used directly. It will be inherited by the specific operation's classes

class Operation():
    """
    no docstring
    """
    #Class Attributes
    #WARNING None

    #Defining Constructor
    def __init__(self, input_nodes=[]):
        #object Operation has two nodes
        self.input_nodes = input_nodes
        self.output_nodes = []

        #every operation is appended to the output node
        for node in input_nodes:
            node.output_nodes.append(self)

        #global var declared in Graph Class
        _default_graph.operation.append(self)

    #Compute method will be overwritten by the operations' classes
    def compute(self):
        pass

In [13]:
#Single operation classes

#Add inheriths methods from Operation
class add(Operation):

    #Add takes in x and y and adds them
    def __init__(self, x, y):
        #super call to receive arguments from the inherithed class
        super().__init__([x,y])


    #Overwriting Compute method from Operation Class
    def compute(self, x_var, y_var):
        self.inputs = [x_var, y_var]
        return x_var + y_var

#Same concepts applies to the classes below
#######################################################
class multiply(Operation):

    def __init__(self, x, y):
        super().__init__([x,y])

    def compute(self, x_var, y_var):
        self.inputs = [x_var, y_var]
        return x_var * y_var

#######################################################
class matmul(Operation):

    def __init__(self, x, y):
        super().__init__([x,y])

    def compute(self, x_var, y_var):
        self.inputs = [x_var, y_var]
        return x_var.dot(y_var)

#######################################################


#### Building Graph and Placeholder for variable

In [14]:
#Graph class will connect placeholder, variables and operations
class Graph():

    def __init__(self):
        self.operation = []
        self.placeholders = []
        self.variables = []

    def set_as_default(self):
        global _default_graph
        _default_graph = self

In [15]:
#Placeholder class will hold the values of the nodes
class Placeholder():

    def __init__(self):

        self.output_nodes = []
        _default_graph.placeholders.append(self)

#### Variable class will hold the  weights

In [16]:
#Changeable parameters of the graph ( the weights)
class Variable():

    def __init__(self, initial_value=None):

        self.value = initial_value
        self.output_nodes = []

        _default_graph.variables.append(self)

#### Flow control and linker classes

In [17]:
#Provides flow control for the operations
class Session():

    #feed_dict maps placeholders to input values
    # later they'll feed batches of data to the network
    def run(self, operation, feed_dict={}):

        nodes_postorder = traverse_postorder(operation)

        for node in nodes_postorder:

            #IF NOT A Placeholder
            if type(node) == Placeholder:
                node.output = feed_dict[node]
            #And NOT an operation
            elif type(node) == Variable:
                node.output = node.value
            #Then EXECUTE the operation for all the nodes
            else:
                node.inputs = [input_node.output for input_node in node.input_nodes]
                node.output = node.compute(*node.inputs) # * to handle multiple args

            if type(node.output) == list:
                #converting the list to numpy arrays
                node.output = np.array(node.output)

        return operation.output

#Declaring session object
sess = Session()


In [18]:
#This function will define the order of the operations (tree traversal order)
def traverse_postorder(operation):
    """
    PortOrder Traversal of Nodes. Basically makes sure computations
    are done in the correct order (Ax first, then Ax + b).
    """

    nodes_postorder = []
    def recurse(node):
        if isinstance(node, Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
        nodes_postorder.append(node)

    recurse(operation)
    return nodes_postorder

Now that the framework is complete, let's run the operations

In [20]:
#assigning the variable to a placeholder object
x = Placeholder() 

NameError: name '_default_graph' is not defined

Randomly using the classes without creating a session and a graph will produce an error

#### Code to be run

In [62]:
sess = Session()
#Declaring a graph object for a matrix multiplication
g = Graph()
#setting default graph
g.set_as_default()

In [63]:
#Passing the variables
A = Variable([[10,20],[30,40]])
b = Variable([1,1])

In [64]:
#Allocating the placeholder
x = Placeholder()
#Assigning the value of the operation to z
z = matmul(A,x)

In [65]:
sess.run(operation = z, feed_dict={x:10})

array([[100, 200],
       [300, 400]])